In [52]:
# load tidy dataset

import pandas as pd
import os.path
import numpy as np
from itertools import combinations
import glob

#import numpy as np
#import loadTidyCSV.py

def loadTidy(tidyData): 
    assert os.path.isfile(tidyData), "desired file does not exist" 
    df = pd.read_csv(tidyData, header = None, names = ["date", "animal", "session", "trial", "stimulus", "neuronID", "timePt", "CaSignal"])
    return df 

# tidy csv file and dir (use makeTidy_Anderson.m to convert Ann's structure to csv)
tidyDataDir = '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/'
tidyDataFileTemplate = 'mouse'
tidyDataFileExt = '.csv'

# get all input files you want to add to the same dataset
dataFiles = np.sort(glob.glob(\
    "/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse*.csv"))
print("data files: \n", dataFiles)


print("loading and appending to prior pandas data frame")
dataLst = []
for file in dataFiles:
    print(file)
    dataLst.append(loadTidy(file))
df = pd.concat(dataLst)

print('finished loading')

############# ALL DATA STORED HERE IN DF
    

data files: 
 [ '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse4.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse5.csv']
loading and appending to prior pandas data frame
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnal

In [53]:
## survey/search data to prepare for split operation
metaStrs = [['dates','date'],['animals','animal'],['sessions','session'],['maxTrials','trial'],['stimuli','stimulus']]

## this dictionary holds useful info regarding the range of inputs to loop/search over subsequently
metaDct = {}
for a,b in metaStrs:
    print(a,b)
    metaDct[a] = np.unique(df[b].tolist())
print(metaDct)


dates date
animals animal
sessions session
maxTrials trial
stimuli stimulus
{'dates': array(['2017_05_00'], 
      dtype='<U10'), 'animals': array([1, 3, 4, 5, 7]), 'sessions': array([1, 2, 3]), 'maxTrials': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21]), 'stimuli': array(['USS', 'baseline', 'female', 'male', 'mineral oil odor',
       'peanut odor', 'pred odor', 'rat', 'tone', 'toy'], 
      dtype='<U16')}


In [19]:
testInds = (df['animal']==1) & (df['session']==1) & (df['stimulus']=='USS') & (df['trial']==5)
print(np.sum(testInds))

26808


In [49]:
### get all pairs of stimuli
stimCmbTpl = tuple(combinations(metaDct['stimuli'],2)) 

######### MAIN LOOP ##########
for (stimA, stimB) in stimCmbTpl:
    
    ## get all data for both trial types
    indsBoth = (df['stimulus']==stimA) | (df['stimulus']==stimB)
    df_bothStimuli = df[indsBoth]

    ## select data by animals and sessions
    for animal in metaDct['animals']:
        print('animal: ', animal)
        for session in metaDct['sessions']:
            print('session:', session)
            inds_animalSession = (df_bothStimuli['animal'] == animal) & (df_bothStimuli['session'] == session)
            df_animalSession = df_bothStimuli[inds_animalSession]
            
            print(stimA,stimB)
            ## get both stim
            df_anmlSessStimA = df_animalSession[df_animalSession['stimulus'] == stimA]
            df_anmlSessStimB = df_animalSession[df_animalSession['stimulus'] == stimB]
            
            ## get trial numbers of each stimuli's presentations 
            trials_stimA = np.unique(df_anmlSessStimA['trial'].tolist())
            trials_stimB = np.unique(df_anmlSessStimB['trial'].tolist())
            print(trials_stimA,trials_stimB)
        
            ## CHECK BOTH PRESENTATIONS (USU 2 OR 3)
            for trialA,trialB in zip(trials_stimA,trials_stimB): 
                
                
                
                
                
                
                
                
                
                try:                    
                    assert trialA != [] and trialB != [], "no fair comparison available (trial not conducted for one of the stimulus types)"
#                     print("fair comparisons exist --> proceeding")
                except:
                    print("expection")
                    print(trialA)
                    print(trialB)
                    numTimePts_thisTrial_stimA = np.unique(df_anmlSessStimA[df_anmlSessStimA['trial'] == str(trialA)]).tolist()
                    numTimePts_thisTrial_stimB = np.unique(df_anmlSessStimB[df_anmlSessStimA['trial'] == str(trialB)]).tolist()
                    print(numTimePts_thisTrial_stimA,numTimePts_thisTrial_stimB)
                    break    
            
            indsB = (df_bothStimuli['stimulus'] == stimB) & (df_bothStimuli['session'] == session)
            numA = np.sum(indsA)
            numB = np.sum(indsB)
            print(numA,numB)
    #         print(df_bothStimuli[indsA],df_bothStimuli[indsB])
            print("sum",np.sum(indsA))
        print('\n')

    ## (REALLY? SAME RANDOMIZATION FOR EVERY ANIMAL?)


animal:  1
session: 1
USS baseline
[ 5 10 16] [ 1  8 15]
expection
5
1


/home/orthogonull/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [ ]:
np.unique(df_anmlSessStimA[df_anmlSessStimA['trial'] == trialA].tolist())

In [88]:
print(metaDct['animals'])
print(metaDct['stimuli'])



[1]
['USS' 'baseline' 'female' 'male' 'rat' 'tone' 'toy']


In [91]:
# this works but is against the whole point of doing pandas in the first place

for date in metaDct['dates']:
    for animal in metaDct['animals']:
        for session in metaDct['sessions']:
            for stimulus in metaDct['stimuli']:
                print(date, animal, session, stimulus)
                
    

2017_05_00 1 1 USS
2017_05_00 1 1 baseline
2017_05_00 1 1 female
2017_05_00 1 1 male
2017_05_00 1 1 rat
2017_05_00 1 1 tone
2017_05_00 1 1 toy
2017_05_00 1 2 USS
2017_05_00 1 2 baseline
2017_05_00 1 2 female
2017_05_00 1 2 male
2017_05_00 1 2 rat
2017_05_00 1 2 tone
2017_05_00 1 2 toy
2017_05_00 1 3 USS
2017_05_00 1 3 baseline
2017_05_00 1 3 female
2017_05_00 1 3 male
2017_05_00 1 3 rat
2017_05_00 1 3 tone
2017_05_00 1 3 toy


In [17]:

# gb = df.groupby(['stimulusType']).get_group('rat')

stimGrouped = df.groupby('stimulusType')
ratGrpd = stimGrouped.get_group('rat')

print(ratGrpd)

print('reached end of file')

               date  animalNum  sessionNum  trialNum stimulusType  neuronID  \
36240    2017_05_00          1           1         3          rat         1   
36241    2017_05_00          1           1         3          rat         1   
36242    2017_05_00          1           1         3          rat         1   
36243    2017_05_00          1           1         3          rat         1   
36244    2017_05_00          1           1         3          rat         1   
36245    2017_05_00          1           1         3          rat         1   
36246    2017_05_00          1           1         3          rat         1   
36247    2017_05_00          1           1         3          rat         1   
36248    2017_05_00          1           1         3          rat         1   
36249    2017_05_00          1           1         3          rat         1   
36250    2017_05_00          1           1         3          rat         1   
36251    2017_05_00          1           1         3

In [ ]:
gb = df.groupby('stimulusType')

df_means = gb.apply(np.mean)
df_means

In [ ]:
for stimType in gb

In [141]:
ratData = df.loc[df['stimulusType'] == 'rat',:]
ussData = df.loc[df['stimulusType'] == 'USS',:]

KeyError: 'stimulusType'

In [142]:
print(len(ratData))
print(len(ussData))

NameError: name 'ratData' is not defined

In [27]:
print(ussData)

# slicing
df_big_force = df.loc[df['impact force (mN)'] > 1000, :]

df = pd.concat((df_low, df_high), axis=1)

# Specify indices we want (note parentheses holding each Boolean)
inds = (df['food density'] == 'high') & (df['cross-sectional area (sq micron)'] > 2000)

# Pull out areas
df.loc[inds, 'cross-sectional area (sq micron)']

df.corr()

# Rename the impact force column
df = df.rename(columns={'impact force (mN)': 'impf'})



# Write out DataFrame
df.to_csv('xa_combined.csv', index=False)

               date  animalNum  sessionNum  trialNum stimulusType  neuronID  \
90432    2017_05_00          1           1         5          USS         1   
90433    2017_05_00          1           1         5          USS         1   
90434    2017_05_00          1           1         5          USS         1   
90435    2017_05_00          1           1         5          USS         1   
90436    2017_05_00          1           1         5          USS         1   
90437    2017_05_00          1           1         5          USS         1   
90438    2017_05_00          1           1         5          USS         1   
90439    2017_05_00          1           1         5          USS         1   
90440    2017_05_00          1           1         5          USS         1   
90441    2017_05_00          1           1         5          USS         1   
90442    2017_05_00          1           1         5          USS         1   
90443    2017_05_00          1           1         5

In [16]:




inds = df.loc(df.animalNum == 1)
df[inds]

KeyError: <pandas.core.indexing._LocIndexer object at 0x7fedd38aadd8>